In [2]:
import time

from bs4 import BeautifulSoup
import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [26]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Volumes/UsersHome/linglv/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


<IPython.core.display.Javascript object>

In [29]:
url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Georgia%20Tech%2C%20Atlanta%2C%20GA&ns=1&l=g%3A-84.41276550292969%2C33.76816006007425%2C-84.38718795776367%2C33.78956253387451"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")
time.sleep(3)


result = []
while True:
    eles = soup.find_all(
        "div", class_=["container__09f24__sxa9-", "hoverable__09f24__3HkY2"]
    )
    for ele in eles:
        name = ele.find_all("div", class_="businessName__09f24__3Ml2X")[0].text
        reviews = None
        price = None
        restauran_type = []
        star = None
        print(name)
        try:
            reviews = ele.find_all(
                "span", class_="reviewCount__09f24__3GsGY css-e81eai"
            )[0].text
        except:
            pass

        try:
            price = ele.find_all("span", class_="priceRange__09f24__2GspP css-e81eai")[
                0
            ].text
        except:
            pass

        try:
            types = ele.find_all("p", class_=["text__09f24__2NHRu", "css-1hx6l2b"])
            l = len(types)
            for i in range(l):
                restauran_type.append(types[i].text)
        except Exception as e:
            print(e)
            pass

        try:
            star = ele.find_all("div", class_="i-stars__09f24___sZu0")[0]["aria-label"]
        except:
            pass
        d = {
            'name': name,
            'reviews': reviews,
            'price': price,
            'star': star,
            'restauran_type': restauran_type
        }
        result.append(d)
    try:
        browser.find_by_css(".next-link.navigation-button__09f24__1EzzD.css-166la90").click()
        time.sleep(3)
        print('next page')
        html = browser.html
        soup = BeautifulSoup(html, "html.parser")
    except:
        print('end of the list')
        break


Moe’s Southwest Grill
QDOBA Mexican Eats
1. Food Terminal - Atlanta
2. There on Fifth
3. Rock Steady - Atlanta
4. The Optimist
5. Choong Man Chicken - Georgia Tech
6. Velvet Taco
7. Pijiu Belly
8. Forza Storico
9. Wagaya - Westside
10. McCray’s Tavern Midtown
Star of India
next page
Star of India
Forza Storico
11. bartaco
12. Rowdy Tiger Whiskey Bar & Kitchen
13. JCT Kitchen & Bar
14. Le Fat
15. Spice House Midtown
16. Redbird
17. E Ramen +
18. Aziza
19. Antico Pizza
20. Barcelona Westside Ironworks
QDOBA Mexican Eats
next page
Star of India
Forza Storico
21. Marcel
22. Tannour Mediterranean Grill
23. Momonoki
24. Thumbs Up Diner
25. El Burro Pollo
26. Nagomiya
27. Cheba Hut Toasted Subs
28. Omakase By Yun
29. Little Italy Atlanta
30. Gio’s Chicken Amalfitano
Moe’s Southwest Grill
next page
Star of India
Forza Storico
31. Steamhouse Lounge
32. Cooks & Soldiers
33. Nova Sushi Bar & Asian Bistro II
34. Momo Cafe
35. Snooze, an A.M. Eatery
36. West Egg Café
37. Marlow’s Tavern
38. Oddbird

<IPython.core.display.Javascript object>

In [36]:
df = pd.DataFrame(result)
df["rating"] = df["star"].str.split(" ").str[0]
df["name"] = df["name"].str.replace("^[0-9]+\.\s", "", regex=True)
df["star"] = df["star"].str.replace("[A-z\s\.]+$", "", regex=True)
df["advertised_restrant"] = df["name"].str.contains("^[0-9]+\.\s", regex=True)
df.loc[df["advertised_restrant"] == False, "advertised_restrant"] = "yes"
df.loc[df["advertised_restrant"] == True, "advertised_restrant"] = "no"
df

,name,reviews,price,star,restauran_type,rating,advertised_restrant
0,Moe’s Southwest Grill,None,None,None,"[Caterers, Tex-Mex, Mexican]",None,yes
1,QDOBA Mexican Eats,5,None,3,"[Mexican, Fast Food]",3,yes
2,Food Terminal - Atlanta,173,$$,4.5,"[Malaysian, Pan Asian, Chinese]",4.5,yes
3,There on Fifth,39,None,5,"[American (New), Cocktail Bars, Chicken Wings]",5,yes
4,Rock Steady - Atlanta,196,$$,4,"[Caribbean, Bars]",4,yes
...,...,...,...,...,...,...,...
212,The Krystal Company,None,None,None,[Restaurants],None,yes
213,Papa John’s Pizza,None,None,None,[Pizza],None,yes
214,Jimmy’s,None,None,None,[Breakfast & Brunch],None,yes
215,Brittain Dining Hall,None,None,None,[Cafeteria],None,yes


<IPython.core.display.Javascript object>

In [38]:
new_df = df.sort_values(by=["advertised_restrant"]).groupby("name").last().reset_index()

<IPython.core.display.Javascript object>

In [39]:
new_df.to_csv("yelp_clean_data.csv")

<IPython.core.display.Javascript object>